In [1]:
import numpy as np
import torch
import pandas as pd
import tensorflow as tf

no_of_people=1
def function():
    data = pd.read_csv("Bank_Personal_Loan_Modelling.csv")
    data.drop(['ID'], axis=1, inplace = True)
    x = data.drop(['Personal Loan'] , axis = 1).values
    y = data['Personal Loan'].values
    x = torch.tensor(x , dtype = torch.float64) 
    y = torch.tensor(y , dtype = torch.float64)
    y = y.to(torch.float64)
    
    from sklearn.model_selection import train_test_split
    x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42, test_size=0.25)
    return x_train , x_test , y_train , y_test

In [2]:
class NN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        ''' Linear1 is the input layer
            Linear2 is the first hidden layer 
            Linear3 is the second hidden layer
        '''
        self.linear1 = torch.nn.Linear(12, 10)
        self.linear2 = torch.nn.Linear(10, 20)
        self.linear3 = torch.nn.Linear(20 , 1)
        self.relu = torch.nn.ReLU()
        self.sigmoid = torch.nn.Sigmoid()
    ''' First you enter linear1(input layer) and then your output from linear1 passes a ReLu and 
        then enter hidden layer1 (linear2); whatever outputs from linear2 enters linear3 (hidden layer2)
        (there is no optimization function between linear2 and linear3 but the output of linear2 are 
        acted on by the weights of linear3 to give a new output)
        Output from linear3 goes through a relu and then a sigmoid. 
        Output of the sigmoid function is the final output'''
    def forward(self, x):
        x = self.linear1(x.float())
        x = self.relu(x.float())
        x = self.linear2(x.float())
        x = self.linear3(x.float())
        x = self.relu(x.float())
        x = self.sigmoid(x.float())
        return x

In [3]:
'''for i in model.parameters():
    print(i)'''

'for i in model.parameters():\n    print(i)'

1. Initialiser a population with the three <br>
2. Generate offsprings <br>
3. Do selection on offspring to get 2 parent individuals <br>
4. Do crossover on parents to get two children <br>
5. Do mutation on the children <br>
6. Add those children to the population <br>
7. Update weights using the fitness function <br>
8. Every 10th epoch apply decay rate <br>

In [4]:
class GeneticOptimizer:
    '''Initialize your model parameters'''
    def __init__(self, model, population_size, mutation , decay ,  inputs  , labels):
        self.model = model
        self.population_size = population_size
        self.mutation = mutation
        self.population = self.init_population() #---CALLING INIT_POPULATION FUNCTION HERE HERE
        self.decay = decay
        self.inputs = inputs
        self.labels = labels

    def init_population(self):
        population = []
        for i in range(self.population_size):
            weights = []
            ''' self.model.parameters are the things you defined under the __init__ of your model class
                basically those self.linear1, self.linear2 defined will be returned
                Since in linear1 there are 12->10 nodes, no.of connections is 12x10 + 1 bias = 121
                In linear2 it is 10->20 nodes, no.of connections is 10x20 + 1 bias = 201
                In linear 3 it is 20->1 nodes, so 20x1 + 1bias = 21 connections
                In totality there will be 121+201+21 = 343 parameters
                The parameters linear1,linear2,linear3 are returned as a tensor like in the previous cell 
                containing 120,200,20 weights.
                Same bias is used for all three layers
                The last tensor returned as a single element in the list in the prev cell output is the bias. '''
            for weight in self.model.parameters(): 
                weights.append(weight.data.numpy())
                ''' Three sets of weights are returned.
                    Those three sets act as one invidivual
                    So, our population has 3 individuals'''
            #print(f"len of weight in iteration {i+1} is {len(weights)} - tensor1,bias1 , tensor2,bias2, tensor3,bias3")
            population.append(weights)
            #print(f"len of population in iteration {i+1} is {len(population)}\n")
        return population

    '''
        Create a new population.
        self.selection is giving the index of the individual with the best and second best fitness score
        self.population finds that element with this index and assigns parent1 and parent2
        self.crossover - crossover between male and female gives us 2 children - child1 and child2
        Do mutation on both the children
        Append the children to the population'''
    def generate_offspring(self, fitness_scores):
        new_population = []
        for _ in range(self.population_size):
            ''' Initially we have 20 people in the population
                We select 2 people with the best fitness score from those 20 ppl
                Assume 1 is male and the other is female
                In each iteration, the same parent produces 2 children
                
                At the end of 20 iterations (Cause 20=population size), we will have 40 children
                These 40 new children will make our new population
                As in 
                These 40 children replace their parent to become the new gen population
                
                This is run 50 epoch times
                So, the 40 people/children population changes in each epoch 
                by adding 2 children 20 times in this functions' 20 iterations
                '''
            parent1_index = self.selection(fitness_scores)
            parent2_index = self.selection(fitness_scores)
            parent1 = self.population[parent1_index] 
            parent2 = self.population[parent2_index]
            child1, child2 = self.crossover(parent1, parent2)
            child1 = self.mutate(child1)
            child2 = self.mutate(child2)
            new_population.append(child1)
            new_population.append(child2)
            
        self.population = new_population
        #print(f"\nnew population has {len(self.population)} no.of people")

    '''Select individuals with the best fitness score and return their indices'''
    def selection(self, fitness_scores):
        cumulative_scores = np.cumsum(fitness_scores)
        total_score = np.sum(fitness_scores)
        rand = np.random.uniform(0, total_score)
        selected_index = np.searchsorted(cumulative_scores, rand)
        return selected_index
    
    '''Male and Female will do crossover and produce 2 children which will be added to
    the new population (next gen)'''
    def crossover(self, male, female):
        random_crossover = np.random.randint(1, len(male))
        child1 = male[:random_crossover] + female[random_crossover:]
        child2 = male[:random_crossover] + female[random_crossover:]
        return child1, child2
    
    '''randomly do mutations on the child so that changes occur in the subsequent generations
    and not the exact same characteristics remain throughout generations'''
    def mutate(self, child):
        for i in range(len(child)):
            if np.random.uniform(0, 1) < self.mutation:
                child[i] += np.random.normal(0, 0.1, child[i].shape)
        return child
    
    def fitness(self, weights):
        global no_of_people
        for i, param in enumerate(self.model.parameters()):
            #print(f"\n\n\na={a}")
            #print(f"weights[{i}] = {weights[i]}")
            param.data = torch.Tensor(weights[i])
        #print(f"no_of_people = {no_of_people}")
        no_of_people+=1
        outputs = self.model(self.inputs)
        loss = loss_fn(outputs.float(), self.labels.reshape([len(self.inputs) , 1]).float())
        return 1 / (loss.item() + 1e-6)
    
    def update_weight(self):
        fitness_scores = [self.fitness(weights) for weights in self.population]
        best_index = np.argmax(fitness_scores)
        best_weights = self.population[best_index]
        for i, param in enumerate(self.model.parameters()):
            param.data = torch.Tensor(best_weights[i])
            
    def decay_mutation(self):
        self.mutation -= (self.decay*self.mutation)

In [5]:
model = NN()
loss_fn = torch.nn.MSELoss() #this is an in-built torch function

x_train, x_test, y_train, y_test = function()
genetic_optimizer = GeneticOptimizer(model, population_size=20, mutation=0.3, 
                                     decay = 0.05, inputs = x_train, labels = y_train)

loss_list = []
for epoch in range(51):
    genetic_optimizer.generate_offspring([])
    genetic_optimizer.update_weight()
    outputs = model(x_train)
    loss = loss_fn(outputs, y_train.reshape([len(x_train) , 1]).float())
    loss_list.append(loss.item())
    loss.backward()
    genetic_optimizer.generate_offspring([])
    genetic_optimizer.update_weight()
    if (epoch%10 == 0):
        print("Epoch" , epoch , " : " , loss.item())
        genetic_optimizer.decay_mutation()

Epoch 0  :  0.9077333211898804
Epoch 10  :  0.25
Epoch 20  :  0.25
Epoch 30  :  0.25
Epoch 40  :  0.25
Epoch 50  :  0.25
